In [64]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial
from itertools import product
from multiprocessing import Pool
from pandas.testing import assert_frame_equal

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [96]:
dir_dict = dict()

dir_dict["data"] = "../data/"
dir_dict["raw"] = os.path.join(dir_dict["data"], "raw")
dir_dict["mid"] = os.path.join(dir_dict["data"], "mid")
dir_dict["clean"] = os.path.join(dir_dict["data"], "clean")
dir_dict["raw_csv"] = os.path.join(dir_dict["raw"], "csv")
dir_dict["raw_json"] = os.path.join(dir_dict["raw"], "json")
dir_dict["html"] = os.path.join(dir_dict["raw"], "html")
dir_dict["completed_html"] = os.path.join(dir_dict["html"], "completed")
dir_dict["upcoming_html"] = os.path.join(dir_dict["html"], "upcoming")
dir_dict["fighterlist_html"] = os.path.join(dir_dict["html"], "fighterlist")
dir_dict["fighters_html"] = os.path.join(dir_dict["html"], "fighters")
dir_dict["completed_eventlist_html"] = os.path.join(dir_dict["completed_html"], "eventlist")
dir_dict["completed_events_html"] = os.path.join(dir_dict["completed_html"], "events")
dir_dict["completed_fights_html"] = os.path.join(dir_dict["completed_html"], "fights")
dir_dict["upcoming_eventlist_html"] = os.path.join(dir_dict["upcoming_html"], "eventlist")
dir_dict["upcoming_events_html"] = os.path.join(dir_dict["upcoming_html"], "events")
dir_dict["upcoming_fights_html"] = os.path.join(dir_dict["upcoming_html"], "fights")

In [97]:
for folderpath in dir_dict.values():
    os.makedirs(folderpath, exist_ok=True)

In [5]:
filepath = os.path.join(dir_dict["mid"], "completed_fights.csv")
df = pd.read_csv(filepath)

/tmp/ipykernel_254740/2262833221.py:2: DtypeWarning: Columns (217,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [6]:
df.shape

(6875, 257)

In [7]:
df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,Time format,Referee,Details,Fighter1_Overall_KD,Fighter2_Overall_KD,Fighter1_Round1_KD,Fighter2_Round1_KD,Fighter1_Round2_KD,Fighter2_Round2_KD,Fighter1_Overall_Sig. str.,Fighter2_Overall_Sig. str.,Fighter1_Round1_Sig. str.,Fighter2_Round1_Sig. str.,Fighter1_Round2_Sig. str.,Fighter2_Round2_Sig. str.,Fighter1_Overall_Sig. str. %,Fighter2_Overall_Sig. str. %,Fighter1_Round1_Sig. str. %,Fighter2_Round1_Sig. str. %,Fighter1_Round2_Sig. str. %,Fighter2_Round2_Sig. str. %,Fighter1_Overall_Total str.,Fighter2_Overall_Total str.,Fighter1_Round1_Total str.,Fighter2_Round1_Total str.,Fighter1_Round2_Total str.,Fighter2_Round2_Total str.,Fighter1_Overall_Td,Fighter2_Overall_Td,Fighter1_Round1_Td,Fighter2_Round1_Td,Fighter1_Round2_Td,Fighter2_Round2_Td,Fighter1_Overall_Td %,Fighter2_Overall_Td %,Fighter1_Round1_Td %,Fighter2_Round1_Td %,Fighter1_Round2_Td %,Fighter2_Round2_Td %,Fighter1_Overall_Sub. att,Fighter2_Overall_Sub. att,Fighter1_Round1_Sub. att,Fighter2_Round1_Sub. att,Fighter1_Round2_Sub. att,Fighter2_Round2_Sub. att,Fighter1_Overall_Rev.,Fighter2_Overall_Rev.,Fighter1_Round1_Rev.,Fighter2_Round1_Rev.,Fighter1_Round2_Rev.,Fighter2_Round2_Rev.,Fighter1_Overall_Ctrl,Fighter2_Overall_Ctrl,Fighter1_Round1_Ctrl,Fighter2_Round1_Ctrl,Fighter1_Round2_Ctrl,Fighter2_Round2_Ctrl,Fighter1_Overall_Sig. str_SS,Fighter2_Overall_Sig. str_SS,Fighter1_Round1_Sig. str_SS,Fighter2_Round1_Sig. str_SS,Fighter1_Round2_Sig. str_SS,Fighter2_Round2_Sig. str_SS,Fighter1_Overall_Sig. str. %_SS,Fighter2_Overall_Sig. str. %_SS,Fighter1_Round1_Sig. str. %_SS,Fighter2_Round1_Sig. str. %_SS,Fighter1_Round2_Sig. str. %_SS,Fighter2_Round2_Sig. str. %_SS,Fighter1_Overall_Head_SS,Fighter2_Overall_Head_SS,Fighter1_Round1_Head_SS,Fighter2_Round1_Head_SS,Fighter1_Round2_Head_SS,Fighter2_Round2_Head_SS,Fighter1_Overall_Body_SS,Fighter2_Overall_Body_SS,Fighter1_Round1_Body_SS,Fighter2_Round1_Body_SS,Fighter1_Round2_Body_SS,Fighter2_Round2_Body_SS,Fighter1_Overall_Leg_SS,Fighter2_Overall_Leg_SS,Fighter1_Round1_Leg_SS,Fighter2_Round1_Leg_SS,Fighter1_Round2_Leg_SS,Fighter2_Round2_Leg_SS,Fighter1_Overall_Distance_SS,Fighter2_Overall_Distance_SS,Fighter1_Round1_Distance_SS,Fighter2_Round1_Distance_SS,Fighter1_Round2_Distance_SS,Fighter2_Round2_Distance_SS,Fighter1_Overall_Clinch_SS,Fighter2_Overall_Clinch_SS,Fighter1_Round1_Clinch_SS,Fighter2_Round1_Clinch_SS,Fighter1_Round2_Clinch_SS,Fighter2_Round2_Clinch_SS,Fighter1_Overall_Ground_SS,Fighter2_Overall_Ground_SS,Fighter1_Round1_Ground_SS,Fighter2_Round1_Ground_SS,Fighter1_Round2_Ground_SS,Fighter2_Round2_Ground_SS,Fighter1_Round3_KD,Fighter2_Round3_KD,Fighter1_Round3_Sig. str.,Fighter2_Round3_Sig. str.,Fighter1_Round3_Sig. str. %,Fighter2_Round3_Sig. str. %,Fighter1_Round3_Total str.,Fighter2_Round3_Total str.,Fighter1_Round3_Td,Fighter2_Round3_Td,Fighter1_Round3_Td %,Fighter2_Round3_Td %,Fighter1_Round3_Sub. att,Fighter2_Round3_Sub. att,Fighter1_Round3_Rev.,Fighter2_Round3_Rev.,Fighter1_Round3_Ctrl,Fighter2_Round3_Ctrl,Fighter1_Round3_Sig. str_SS,Fighter2_Round3_Sig. str_SS,Fighter1_Round3_Sig. str. %_SS,Fighter2_Round3_Sig. str. %_SS,Fighter1_Round3_Head_SS,Fighter2_Round3_Head_SS,Fighter1_Round3_Body_SS,Fighter2_Round3_Body_SS,Fighter1_Round3_Leg_SS,Fighter2_Round3_Leg_SS,Fighter1_Round3_Distance_SS,Fighter2_Round3_Distance_SS,Fighter1_Round3_Clinch_SS,Fighter2_Round3_Clinch_SS,Fighter1_Round3_Ground_SS,Fighter2_Round3_Ground_SS,Fighter1_Round4_KD,Fighter2_Round4_KD,Fighter1_Round4_Sig. str.,Fighter2_Round4_Sig. str.,Fighter1_Round4_Sig. str. %,Fighter2_Round4_Sig. str. %,Fighter1_Round4_Total str.,Fighter2_Round4_Total str.,Fighter1_Round4_Td,Fighter2_Round4_Td,Fighter1_Round4_Td %,Fighter2_Round4_Td %,Fighter1_Round4_Sub. att,Fighter2_Round4_Sub. att,Fighter1_Round4_Rev.,Fighter2_Round4_Rev.,Fighter1_Round4_Ctrl,Fighter2_Round4_Ctrl,Fighter1_Round4_Sig. str_SS,Fighter2_Round4_Sig. str_SS,Fighter1_Round4_Sig. str. %_

In [8]:
df.isnull().sum()

Event Name            0
Fight ID              0
Fighter1 Status       0
Fighter1 Name         0
Fighter2 Status       0
                     ..
Fighter2 Str. Def     0
Fighter2 TD Avg.      0
Fighter2 TD Acc.      0
Fighter2 TD Def.      0
Fighter2 Sub. Avg.    0
Length: 257, dtype: int64

### Rename columns

In [9]:
df.columns = [col.lower().replace(".","").replace(" ", "_") for col in df.columns]

## Clean columns

In [10]:
def null_unique(df, col_names):
    return pd.concat([df[col_names].isnull().sum(), df[col_names].nunique()], axis=1, keys=["nulls", "nunique"])

In [11]:
class ColumnTracker:
    def __init__(self, init_list):
        self.main_list = init_list
        self.print_num_left()
        
    def remove(self, to_remove):
        if not isinstance(to_remove, list):
            to_remove = [to_remove]
            
        for col in to_remove:
            self.main_list.remove(col)
            
        self.print_num_left()
        
    def list_left(self):
        return self.main_list
    
    def num_left(self):
        return len(self.main_list)
    
    def print_num_left(self):
        print(f"Remaining columns: {len(self.main_list)}")

In [12]:
coltrack = ColumnTracker(list(df.columns))

Remaining columns: 257


### Details cols

In [13]:
col_names = [col for col in df.columns if ("details" in col) and (col != "details")]
extra_details_df = df.loc[df[col_names].notnull().any(axis=1), ["details"] + col_names]
extra_details_df.head()

,details,details:____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________technical_decision,details:____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________point_deducted,details:____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________two_points_deducted,details:____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________points_deducted
91,NaN,Groin Strike to Romanov Dave Ha...,NaN,NaN,NaN
172,NaN,NaN,Kicks to Groin by Tavares Eric ...,NaN,NaN
246,NaN,NaN,Illegal Elbows by Nakamura Glen...,NaN,NaN
326,NaN,NaN,Low Blow by Blanco Richard Bert...,NaN,NaN
475,NaN,NaN,Low Blows by Prangley Cecil Peo...,NaN,NaN


In [14]:
extra_details_df.shape

(37, 5)

In [15]:
df = df.drop(columns=col_names)

In [16]:
coltrack.remove(col_names)

Remaining columns: 253


### ID columns

In [17]:
col_names = [col for col in df.columns if "_id" in col]
col_names

['fight_id', 'event_id', 'fighter1_id', 'fighter2_id']

In [18]:
df[col_names].head()

,fight_id,event_id,fighter1_id,fighter2_id
0,0730c4e6fb247fa1,883922e5cd6d8473,d156513a19acf856,2f181c0467965b98
1,55833bf5054126e2,0ee783aa00e468f0,6f018c039b4d5d80,30a09e43f15f1d75
2,f19a56ecc6c150e3,8788beb528894f33,6c2030e0a143a94b,3591d0d5d382a381
3,0bdea30d7a7322f7,db1f2ed63b54b9a7,5e4eec08896c9423,4d44f37107f01b9b
4,acec5f4c6fee1ae2,8a9c6c4301f6d088,008ea710276c9606,31bb0772f21cabd8


In [19]:
null_unique(df, col_names)

,nulls,nunique
fight_id,0,6875
event_id,0,624
fighter1_id,0,1667
fighter2_id,0,2139


In [20]:
assert_frame_equal(df[col_names], df[col_names].applymap(lambda s: hex(int(str(s), 16))[2:].zfill(16)))

In [21]:
df.loc[:, col_names] = df[col_names].applymap(lambda s: int(str(s), 16))

In [22]:
df[col_names].head()

,fight_id,event_id,fighter1_id,fighter2_id
0,518130453482602401,9815915233220068467,15084333311904774230,3393493124467022744
1,6161834638714152674,1073971802333800688,7998828360305040768,3503974524743785845
2,17409322884500902115,9766265477231365939,7791281096687331659,3860095972428391297
3,855300257531241207,15789390316237535655,6795628409158145059,5567842706126805915
4,12460439050927348450,9987977108652478600,40153134515656198,3583466118772992984


In [23]:
coltrack.remove(col_names)

Remaining columns: 249


#### Category columns

In [24]:
remaining_cols = coltrack.list_left()

In [25]:
df[remaining_cols].head()

,event_name,fighter1_status,fighter1_name,fighter2_status,fighter2_name,bout,method,round,time,time_format,referee,details,fighter1_overall_kd,fighter2_overall_kd,fighter1_round1_kd,fighter2_round1_kd,fighter1_round2_kd,fighter2_round2_kd,fighter1_overall_sig_str,fighter2_overall_sig_str,fighter1_round1_sig_str,fighter2_round1_sig_str,fighter1_round2_sig_str,fighter2_round2_sig_str,fighter1_overall_sig_str_%,fighter2_overall_sig_str_%,fighter1_round1_sig_str_%,fighter2_round1_sig_str_%,fighter1_round2_sig_str_%,fighter2_round2_sig_str_%,fighter1_overall_total_str,fighter2_overall_total_str,fighter1_round1_total_str,fighter2_round1_total_str,fighter1_round2_total_str,fighter2_round2_total_str,fighter1_overall_td,fighter2_overall_td,fighter1_round1_td,fighter2_round1_td,fighter1_round2_td,fighter2_round2_td,fighter1_overall_td_%,fighter2_overall_td_%,fighter1_round1_td_%,fighter2_round1_td_%,fighter1_round2_td_%,fighter2_round2_td_%,fighter1_overall_sub_att,fighter2_overall_sub_att,fighter1_round1_sub_att,fighter2_round1_sub_att,fighter1_round2_sub_att,fighter2_round2_sub_att,fighter1_overall_rev,fighter2_overall_rev,fighter1_round1_rev,fighter2_round1_rev,fighter1_round2_rev,fighter2_round2_rev,fighter1_overall_ctrl,fighter2_overall_ctrl,fighter1_round1_ctrl,fighter2_round1_ctrl,fighter1_round2_ctrl,fighter2_round2_ctrl,fighter1_overall_sig_str_ss,fighter2_overall_sig_str_ss,fighter1_round1_sig_str_ss,fighter2_round1_sig_str_ss,fighter1_round2_sig_str_ss,fighter2_round2_sig_str_ss,fighter1_overall_sig_str_%_ss,fighter2_overall_sig_str_%_ss,fighter1_round1_sig_str_%_ss,fighter2_round1_sig_str_%_ss,fighter1_round2_sig_str_%_ss,fighter2_round2_sig_str_%_ss,fighter1_overall_head_ss,fighter2_overall_head_ss,fighter1_round1_head_ss,fighter2_round1_head_ss,fighter1_round2_head_ss,fighter2_round2_head_ss,fighter1_overall_body_ss,fighter2_overall_body_ss,fighter1_round1_body_ss,fighter2_round1_body_ss,fighter1_round2_body_ss,fighter2_round2_body_ss,fighter1_overall_leg_ss,fighter2_overall_leg_ss,fighter1_round1_leg_ss,fighter2_round1_leg_ss,fighter1_round2_leg_ss,fighter2_round2_leg_ss,fighter1_overall_distance_ss,fighter2_overall_distance_ss,fighter1_round1_distance_ss,fighter2_round1_distance_ss,fighter1_round2_distance_ss,fighter2_round2_distance_ss,fighter1_overall_clinch_ss,fighter2_overall_clinch_ss,fighter1_round1_clinch_ss,fighter2_round1_clinch_ss,fighter1_round2_clinch_ss,fighter2_round2_clinch_ss,fighter1_overall_ground_ss,fighter2_overall_ground_ss,fighter1_round1_ground_ss,fighter2_round1_ground_ss,fighter1_round2_ground_ss,fighter2_round2_ground_ss,fighter1_round3_kd,fighter2_round3_kd,fighter1_round3_sig_str,fighter2_round3_sig_str,fighter1_round3_sig_str_%,fighter2_round3_sig_str_%,fighter1_round3_total_str,fighter2_round3_total_str,fighter1_round3_td,fighter2_round3_td,fighter1_round3_td_%,fighter2_round3_td_%,fighter1_round3_sub_att,fighter2_round3_sub_att,fighter1_round3_rev,fighter2_round3_rev,fighter1_round3_ctrl,fighter2_round3_ctrl,fighter1_round3_sig_str_ss,fighter2_round3_sig_str_ss,fighter1_round3_sig_str_%_ss,fighter2_round3_sig_str_%_ss,fighter1_round3_head_ss,fighter2_round3_head_ss,fighter1_round3_body_ss,fighter2_round3_body_ss,fighter1_round3_leg_ss,fighter2_round3_leg_ss,fighter1_round3_distance_ss,fighter2_round3_distance_ss,fighter1_round3_clinch_ss,fighter2_round3_clinch_ss,fighter1_round3_ground_ss,fighter2_round3_ground_ss,fighter1_round4_kd,fighter2_round4_kd,fighter1_round4_sig_str,fighter2_round4_sig_str,fighter1_round4_sig_str_%,fighter2_round4_sig_str_%,fighter1_round4_total_str,fighter2_round4_total_str,fighter1_round4_td,fighter2_round4_td,fighter1_round4_td_%,fighter2_round4_td_%,fighter1_round4_sub_att,fighter2_round4_sub_att,fighter1_round4_rev,fighter2_round4_rev,fighter1_round4_ctrl,fighter2_round4_ctrl,fighter1_round4_sig_str_ss,fighter2_round4_sig_str_ss,fighter1_round4_sig_str_%_ss,fighter2_round4_sig_str_%_ss,fighter1_round4_head_ss,fighter2_round4_head_ss,fighter1_round4_body_ss,fig

In [26]:
col_names = remaining_cols[:7] + ["referee", "time_format"] + \
            ["weight_class", "location", "fighter1_nickname", "fighter2_nickname",
             "fighter1_stance", "fighter2_stance"]

In [27]:
null_unique(df, col_names)

,nulls,nunique
event_name,0,624
fighter1_status,0,4
fighter1_name,0,1666
fighter2_status,0,4
fighter2_name,0,2138
bout,0,103
method,0,10
referee,32,208
time_format,0,13
weight_class,0,15


In [28]:
df.loc[:, col_names] = df[col_names].astype("category")

In [29]:
coltrack.remove(col_names)

Remaining columns: 234


### Percentage columns

In [30]:
is_pct_col = df.select_dtypes("object").apply(lambda s: s.str.contains("%")).any()
col_names = is_pct_col[is_pct_col == True].index.to_list()

In [31]:
df[col_names].head()

,fighter1_overall_sig_str_%,fighter2_overall_sig_str_%,fighter1_round1_sig_str_%,fighter2_round1_sig_str_%,fighter1_round2_sig_str_%,fighter2_round2_sig_str_%,fighter1_overall_td_%,fighter2_overall_td_%,fighter1_round1_td_%,fighter2_round1_td_%,fighter1_round2_td_%,fighter2_round2_td_%,fighter1_overall_sig_str_%_ss,fighter2_overall_sig_str_%_ss,fighter1_round1_sig_str_%_ss,fighter2_round1_sig_str_%_ss,fighter1_round2_sig_str_%_ss,fighter2_round2_sig_str_%_ss,fighter1_round3_sig_str_%,fighter2_round3_sig_str_%,fighter1_round3_td_%,fighter2_round3_td_%,fighter1_round3_sig_str_%_ss,fighter2_round3_sig_str_%_ss,fighter1_round4_sig_str_%,fighter2_round4_sig_str_%,fighter1_round4_td_%,fighter2_round4_td_%,fighter1_round4_sig_str_%_ss,fighter2_round4_sig_str_%_ss,fighter1_round5_sig_str_%,fighter2_round5_sig_str_%,fighter1_round5_td_%,fighter2_round5_td_%,fighter1_round5_sig_str_%_ss,fighter2_round5_sig_str_%_ss,fighter1_str_acc,fighter1_str_def,fighter1_td_acc,fighter1_td_def,fighter2_str_acc,fighter2_str_def,fighter2_td_acc,fighter2_td_def
0,42%,66%,41%,67%,42%,64%,NaN,0%,NaN,0%,NaN,NaN,42%,66%,41%,67%,42%,64%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44%,52%,14%,82%,55%,44%,44%,50%
1,44%,45%,33%,50%,66%,0%,100%,0%,100%,0%,100%,NaN,44%,45%,33%,50%,66%,0%,43%,50%,100%,NaN,43%,50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50%,54%,51%,56%,39%,63%,31%,63%
2,46%,68%,56%,76%,20%,71%,0%,50%,0%,50%,0%,50%,46%,68%,56%,76%,20%,71%,52%,63%,0%,50%,52%,63%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44%,56%,19%,57%,42%,59%,36%,73%
3,35%,31%,32%,36%,37%,26%,25%,100%,0%,NaN,0%,100%,35%,31%,32%,36%,37%,26%,39%,28%,100%,NaN,39%,28%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46%,59%,50%,83%,42%,60%,32%,57%
4,75%,80%,75%,81%,77%,66%,NaN,0%,NaN,0%,NaN,NaN,75%,80%,75%,81%,77%,66%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48%,55%,0%,66%,60%,26%,33%,0%


Convert strings to percent

In [32]:
df.loc[:,col_names] = df.loc[:,col_names].apply(lambda s: s.str.replace("%","").astype("float") / 100)

In [33]:
coltrack.remove(col_names)

Remaining columns: 190


### Out-of columns

In [39]:
df.shape

(6875, 373)

In [40]:
coltrack.remove(col_names)

Remaining columns: 70


In [41]:
remaining_cols = coltrack.list_left()

In [42]:
df[remaining_cols].head()

,round,time,details,fighter1_overall_kd,fighter2_overall_kd,fighter1_round1_kd,fighter2_round1_kd,fighter1_round2_kd,fighter2_round2_kd,fighter1_overall_sub_att,fighter2_overall_sub_att,fighter1_round1_sub_att,fighter2_round1_sub_att,fighter1_round2_sub_att,fighter2_round2_sub_att,fighter1_overall_rev,fighter2_overall_rev,fighter1_round1_rev,fighter2_round1_rev,fighter1_round2_rev,fighter2_round2_rev,fighter1_overall_ctrl,fighter2_overall_ctrl,fighter1_round1_ctrl,fighter2_round1_ctrl,fighter1_round2_ctrl,fighter2_round2_ctrl,fighter1_round3_kd,fighter2_round3_kd,fighter1_round3_sub_att,fighter2_round3_sub_att,fighter1_round3_rev,fighter2_round3_rev,fighter1_round3_ctrl,fighter2_round3_ctrl,fighter1_round4_kd,fighter2_round4_kd,fighter1_round4_sub_att,fighter2_round4_sub_att,fighter1_round4_rev,fighter2_round4_rev,fighter1_round4_ctrl,fighter2_round4_ctrl,fighter1_round5_kd,fighter2_round5_kd,fighter1_round5_sub_att,fighter2_round5_sub_att,fighter1_round5_rev,fighter2_round5_rev,fighter1_round5_ctrl,fighter2_round5_ctrl,date,fighter1_record,fighter1_height,fighter1_weight,fighter1_reach,fighter1_dob,fighter1_slpm,fighter1_sapm,fighter1_td_avg,fighter1_sub_avg,fighter2_record,fighter2_height,fighter2_weight,fighter2_reach,fighter2_dob,fighter2_slpm,fighter2_sapm,fighter2_td_avg,fighter2_sub_avg
0,2,2:10,Rear Naked Choke,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0:07,0:44,0:07,0:34,0:00,0:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"February 05, 2022",33-18-1 (1 NC),"6'2""",185 lbs.,"75""","May 06, 1986",3.28,3.63,0.06,0.1,20-5-0,"6'2""",185 lbs.,"75""","Dec 28, 1995",3.99,4.01,1.31,1.5
1,3,5:00,Patricia Morse-Jarman ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12:21,1:23,3:38,0:41,4:24,0:22,0.0,0.0,1.0,1.0,0.0,0.0,4:19,0:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"July 11, 2009",30-7-1 (1 NC),"6'0""",170 lbs.,"74""","Feb 24, 1978",2.61,1.65,3.60,0.8,18-9-0,"5'11""",170 lbs.,"74""","Jan 25, 1981",1.71,1.99,1.42,1.2
2,3,5:00,Tony Weeks ...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6:00,4:44,2:31,1:14,1:50,2:15,0.0,0.0,1.0,0.0,0.0,0.0,1:39,1:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"October 29, 2011",24-13-0,"6'2""",185 lbs.,"75""","Nov 20, 1986",3.57,2.96,0.27,0.5,25-12-0,"6'3""",205 lbs.,"78""","Oct 11, 1981",2.22,1.64,1.79,0.6
3,3,5:00,Chris Lee ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2:45,0:15,0:00,0:00,0:00,0:15,0.0,0.0,0.0,0.0,0.0,0.0,2:45,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"March 05, 2016",14-5-0,"6'3""",205 lbs.,"79""","Sep 22, 1989",4.43,2.22,4.89,0.0,10-6-0 (1 NC),"6'0""",205 lbs.,"74""","May 15, 1983",2.50,2.41,2.46,1.4
4,2,0:46,Punches to Head From Back Control,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3:26,0:55,3:00,0:55,0:26,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"October 23, 2021",11-2-0,"5'6""",125 lbs.,"69""","Jul 17, 1997",7.25,5.04,0.00,0.0,11-4-0,"5'6""",125 lbs.,"70""","Jun 05, 1996",4.29,10.26,1.40,0.0


### Float columns

In [ ]:
col_names = df[remaining_cols].select_dtypes("float").columns.to_list()

In [ ]:
df[col_names].head()

,fighter1_overall_kd,fighter2_overall_kd,fighter1_round1_kd,fighter2_round1_kd,fighter1_round2_kd,fighter2_round2_kd,fighter1_overall_sub_att,fighter2_overall_sub_att,fighter1_round1_sub_att,fighter2_round1_sub_att,fighter1_round2_sub_att,fighter2_round2_sub_att,fighter1_overall_rev,fighter2_overall_rev,fighter1_round1_rev,fighter2_round1_rev,fighter1_round2_rev,fighter2_round2_rev,fighter1_round3_kd,fighter2_round3_kd,fighter1_round3_sub_att,fighter2_round3_sub_att,fighter1_round3_rev,fighter2_round3_rev,fighter1_round4_kd,fighter2_round4_kd,fighter1_round4_sub_att,fighter2_round4_sub_att,fighter1_round4_rev,fighter2_round4_rev,fighter1_round5_kd,fighter2_round5_kd,fighter1_round5_sub_att,fighter2_round5_sub_att,fighter1_round5_rev,fighter2_round5_rev,fighter1_slpm,fighter1_sapm,fighter1_td_avg,fighter1_sub_avg,fighter2_slpm,fighter2_sapm,fighter2_td_avg,fighter2_sub_avg
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.28,3.63,0.06,0.1,3.99,4.01,1.31,1.5
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.61,1.65,3.60,0.8,1.71,1.99,1.42,1.2
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.57,2.96,0.27,0.5,2.22,1.64,1.79,0.6
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.43,2.22,4.89,0.0,2.50,2.41,2.46,1.4
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25,5.04,0.00,0.0,4.29,10.26,1.40,0.0


In [ ]:
df.loc[:, col_names] = df[col_names].astype("float")

In [46]:
coltrack.remove(col_names)

Remaining columns: 26


### Time columns

In [ ]:
is_time_col = df.select_dtypes("object").apply(lambda s: s.str.contains(r"\d+:\d+")).any()
col_names = is_time_col[is_time_col == True].index.to_list()

In [ ]:
def seconds_extrator(row):
    return row[0]*60 + row[1]

In [ ]:
for col in tqdm(col_names):
    df[f"{col}_seconds"] = df[col].str.extract(r"(\d+):(\d+)").astype("float").apply(seconds_extrator, axis=1)

100%|██████████| 13/13 [00:01<00:00,  7.18it/s]


In [51]:
df[[col for col in df.columns if "seconds" in col]].head()

,time_seconds,fighter1_overall_ctrl_seconds,fighter2_overall_ctrl_seconds,fighter1_round1_ctrl_seconds,fighter2_round1_ctrl_seconds,fighter1_round2_ctrl_seconds,fighter2_round2_ctrl_seconds,fighter1_round3_ctrl_seconds,fighter2_round3_ctrl_seconds,fighter1_round4_ctrl_seconds,fighter2_round4_ctrl_seconds,fighter1_round5_ctrl_seconds,fighter2_round5_ctrl_seconds
0,130.0,7.0,44.0,7.0,34.0,0.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
1,300.0,741.0,83.0,218.0,41.0,264.0,22.0,259.0,20.0,NaN,NaN,NaN,NaN
2,300.0,360.0,284.0,151.0,74.0,110.0,135.0,99.0,75.0,NaN,NaN,NaN,NaN
3,300.0,165.0,15.0,0.0,0.0,0.0,15.0,165.0,0.0,NaN,NaN,NaN,NaN
4,46.0,206.0,55.0,180.0,55.0,26.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
coltrack.remove(col_names)

Remaining columns: 13


### Date columns

In [53]:
col_names = ["date", "fighter1_dob", "fighter2_dob"]

In [54]:
df[col_names].isnull().sum()

date              0
fighter1_dob     47
fighter2_dob    145
dtype: int64

In [55]:
for col in col_names:
    df[col] = pd.to_datetime(df[col], format="%B %d, %Y", errors="coerce")\
                    .fillna(pd.to_datetime(df[col], format="%b %d, %Y", errors="coerce"))

In [56]:
df[col_names].isnull().sum()

date              0
fighter1_dob     47
fighter2_dob    145
dtype: int64

In [57]:
coltrack.remove(col_names)

Remaining columns: 10


### Record columns

In [58]:
col_names = [col for col in df.columns if "record" in col]

In [59]:
df[col_names].isnull().sum()

fighter1_record    0
fighter2_record    0
dtype: int64

In [60]:
for col in col_names:
    pattern = f"(?P<{col}_wins>\d+)-(?P<{col}_losses>\d+)-(?P<{col}_draws>\d+)(?:\s\((?P<{col}_ncs>\d+) NC\))?"
    df = pd.concat([df, df[col].str.extract(pattern)], axis=1)

In [61]:
df.iloc[:,-8:].isnull().sum()

fighter1_record_wins         0
fighter1_record_losses       0
fighter1_record_draws        0
fighter1_record_ncs       5504
fighter2_record_wins         0
fighter2_record_losses       0
fighter2_record_draws        0
fighter2_record_ncs       5660
dtype: int64

In [62]:
df = df.drop(columns=col_names)

In [63]:
coltrack.remove(col_names)

Remaining columns: 8


### Height columns

In [66]:
col_names = [f"fighter{i}_height" for i in [1, 2]]
col_names

['fighter1_height', 'fighter2_height']

In [70]:
null_unique(df, col_names)

,nulls,nunique
fighter1_height,4,23
fighter2_height,15,23


In [77]:
for col in col_names:
    df[f"{col}_inches"] = \
            df[col].str.extract("(\d+)'(\d+)\"").astype("float").apply(lambda row: row[0]*12 + row[1], axis=1)

In [78]:
df = df.drop(columns=col_names)

In [79]:
coltrack.remove(col_names)

Remaining columns: 6


### Reach columns

In [80]:
col_names = [f"fighter{i}_reach" for i in [1, 2]]
col_names

['fighter1_reach', 'fighter2_reach']

In [81]:
null_unique(df, col_names)

,nulls,nunique
fighter1_reach,364,27
fighter2_reach,842,27


In [82]:
for col in col_names:
    df[f"{col}_inches"] = \
            df[col].str.extract("(\d+)\"").astype("float")

In [83]:
df = df.drop(columns=col_names)

In [84]:
coltrack.remove(col_names)

Remaining columns: 4


### Weight columns

In [85]:
col_names = [f"fighter{i}_weight" for i in [1, 2]]
col_names

['fighter1_weight', 'fighter2_weight']

In [86]:
null_unique(df, col_names)

,nulls,nunique
fighter1_weight,2,59
fighter2_weight,13,69


In [87]:
for col in col_names:
    df[f"{col}_lbs"] = \
            df[col].str.extract("(\d+) lbs").astype("float")

In [88]:
df = df.drop(columns=col_names)

In [89]:
coltrack.remove(col_names)

Remaining columns: 2


### Remaining columns

In [90]:
df["round"] = df["round"].astype("uint8")

In [91]:
df["details"] = df["details"].astype("string")

In [92]:
col_names = ["round", "details"]

In [93]:
coltrack.remove(col_names)

Remaining columns: 0


### Exporting

In [95]:
df.shape

(6875, 392)

In [98]:
filepath = os.path.join(dir_dict["clean"], "completed_fights.pkl")
df.to_pickle(filepath)

In [99]:
tmp = pd.read_pickle(filepath)

In [100]:
assert_frame_equal(df, tmp)